# Linear probe submission

This notebook uses the **WE-JEPA** framework to pretrain on the `tsbpp__fall2025_deep_learning` dataset, fine-tune a linear probe on **CUB-200**, and generate the Kaggle submission CSV.

Set the dataset roots and toggle the `RUN_PRETRAINING` flag to control whether pretraining runs locally or a pre-computed checkpoint is reused.

In [1]:
from pathlib import Path

import torch

from wejepa.config import IJepaConfig, hf224_config
from wejepa.train.pretrain import launch_pretraining
from wejepa.train.finetune import (
    FinetuneConfig,
    create_submission_csv,
    train_linear_probe,
)


ImportError: cannot import name 'create_submission_csv' from 'wejepa.train.finetune' (/home/long/PhD/Coursework/Deep_Learning/Project/Code/ijepa/src/wejepa/train/finetune.py)

## 1. Pretrain WE-JEPA on the SSL dataset

The model is configured for the `tsbpp__fall2025_deep_learning` train split. Update `data_root` to point to the local cache or mount point that stores the Hugging Face dataset.

In [ ]:

# toggle to run pretraining locally; set to False to reuse an existing checkpoint
RUN_PRETRAINING = False

pretrain_cfg = hf224_config()
pretrain_cfg.data.dataset_root = Path("data/tsbpp_cache").as_posix()
pretrain_cfg.optimizer.epochs = 5
pretrain_cfg.hardware.output_dir = "outputs/ijepa"
pretrain_cfg.hardware.checkpoint_every = 1

if RUN_PRETRAINING:
    launch_pretraining(pretrain_cfg)

pretrained_checkpoint = Path(pretrain_cfg.hardware.output_dir) / f"ijepa_epoch_{pretrain_cfg.optimizer.epochs:04d}.pt"
pretrained_checkpoint


## 2. Fine-tune a linear probe on CUB-200

We clone the pretraining configuration to keep the same model definition, switch the dataset to CUB-200, and train a frozen-backbone linear classifier.

In [ ]:

finetune_cfg = FinetuneConfig(
    ijepa=IJepaConfig.from_dict(pretrain_cfg.to_dict()),
    batch_size=128,
    epochs=10,
    learning_rate=1e-3,
    weight_decay=0.0,
    num_classes=200,
    checkpoint_path=str(pretrained_checkpoint),
)

finetune_cfg.ijepa.data.dataset_name = "cub200"
finetune_cfg.ijepa.data.dataset_root = Path("data/cub200_kaggle").as_posix()
finetune_cfg.ijepa.data.image_size = 96
finetune_cfg.ijepa.data.train_batch_size = finetune_cfg.batch_size
finetune_cfg.ijepa.data.eval_batch_size = 256

linear_probe = train_linear_probe(finetune_cfg)
linear_probe


## 3. Build the Kaggle submission

Generate the `filename,label` CSV using the trained probe. The submission loader comes from the WE-JEPA CUB-200 helpers, so the filenames already match the prepared competition format.

In [ ]:

submission_dir = Path("outputs/submissions")
submission_path = submission_dir / "cub200_linear_probe.csv"

# ensure the model is on the right device for inference
linear_probe.eval()
linear_probe.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

submission_csv = create_submission_csv(linear_probe, finetune_cfg.ijepa, submission_path)
submission_csv
